In [1]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List

In [ ]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int


class MazeRouter:
    NUM_LAYERS = 2

    def __init__(self,
                 in_path: str,
                 out_path: str,  #Check if needed later on
                 routing: str = "routing.txt",
                 dimensions: str = "dimensions.txt"):
        self.in_path = in_path
        self.out_path = out_path
        self.routing = routing
        self.dimensions = dimensions

        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0



        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []
        #The nets are further broken down into those which are on the same layer and those which span different layers. 
        self.same_layer_nets: List[List[Coord]]  = []
        self.multi_layer_nets: List[List[Coord]] = []

        self.clear_routings()
        self.load_config()
        self.save_dimensions()

    def clear_routings(self):
        with open(self.routing, 'w'):
            pass
        print("clear_routing Successful")

    def load_config(self):
        with open(self.in_path, 'r') as file:
            header = file.readline().strip()
            c, r, b, v = map(int, header.split(','))
            self.cols, self.rows = c, r
            self.Direction, self.via = b, v

            self.grid = []
            for k in range(self.NUM_LAYERS):
                layer = []
                for j in range(self.rows):
                    row = []
                    for i in range(self.cols):
                        row.append(Cell(CellType.EMPTY))
                    layer.append(row)
                self.grid.append(layer)

            for line in file:
                line = line.strip()
                if line.startswith("OBS"):
                    self.add_obstacle(line)
                elif line.startswith("net"):
                    self.add_net(line)

    def save_dimensions(self):
        with open(self.dimensions, 'w') as file:
            file.write(f"{self.cols} {self.rows}\n")


    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print("Incorrect Format.", file=sys.stderr)
            return
        layer, col, row = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK
        else:
            print("Obstacle out of Range", file=sys.stderr)

    def add_net(self, text: str):
        path = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            layer, col, row = map(int, m.groups())
            if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
                path.append(Coord(row, col, layer))
            else:
                print("Coordinates out of Range", file=sys.stderr)
                return
        self.nets.append(path)
        
        
    

In [ ]:
def set_SourceAndTarget(self, net_coordinates):
    for index, coord in enumerate(net_coordinates):
        if index == 0:
            self.grid[coord.layer][coord.row][coord.col].kind = CellType.SOURCE
        else:
            self.grid[coord.layer][coord.row][coord.col].kind = CellType.TARGET
            
def categorize_nets(self):

        self.same_layer_nets.clear()
        self.multi_layer_nets.clear()

        for net_idx, net in enumerate(self.nets):
            layers_in_net = {coord.layer for coord in net}
            if len(layers_in_net) == 1:
                self.same_layer_nets.append(net)
                print(f"Net {net_idx} → SAME layer {layers_in_net.pop()}")
            else:
                self.multi_layer_nets.append(net)
                print(f"Net {net_idx} → MULTI layers {layers_in_net}")
            


def back_propagate(self, target: Coord) -> List[Coord]:
    
    route: List[Coord] = [target]
    current = Coord(target.row, target.col, target.layer)
    via_used = False
    max_iters = self.rows * self.cols * 2
    iters = 0


    while self.grid[current.layer][current.row][current.col].kind != CellType.SOURCE:
        if iters > max_iters:
            print("aborting", file=sys.stderr)
            break
        iters += 1

        best_cost = float('inf')
        best_dir = None  

        cell = self.grid[current.layer][current.row][current.col]

      
        if cell.kind == CellType.VIA and not via_used:
            via_used = True

            cell.kind = CellType.EMPTY
            current = Coord(current.row, current.col, (current.layer + 1) % self.NUM_LAYERS)
            route.insert(0, Coord(current.row, current.col, current.layer))
            self.via_count += 1
            continue

        directions = [
            ('up',    (current.row - 1, current.col, current.layer)),
            ('down',  (current.row + 1, current.col, current.layer)),
            ('left',  (current.row, current.col - 1, current.layer)),
            ('right', (current.row, current.col + 1, current.layer)),
        ]

        for dir_name, (r, c, lay) in directions:
    
            if 0 <= r < self.rows and 0 <= c < self.cols:
                neighbor = self.grid[lay][r][c]
                if neighbor.kind == CellType.SOURCE:
                    current = Coord(r, c, lay)
                    route.insert(0, current)
                    return route
                if neighbor.kind not in (CellType.BLOCK, CellType.SOURCE) and neighbor.cost > 0:
                    if neighbor.cost < best_cost:
                        best_cost = neighbor.cost
                        best_dir = (r, c, lay)


        if best_dir:
            current = Coord(*best_dir)
            route.insert(0, Coord(current.row, current.col, current.layer))
            continue


        break

    return route



In [3]:
def main():
    input_file = input("Enter input file name: ").strip()
    output_file = input("Enter output file name: ").strip()

    router = MazeRouter(input_file, output_file)

    print("\nRouter initialized")
    print(f"Maze dimensions: {router.rows} x {router.cols}")
    print(f"Direction penalty: {router.Direction}")
    print(f"Via penalty: {router.via}")

if __name__ == "__main__":
    main()


clear_routing Successful

Router initialized
Maze dimensions: 8 x 8
Direction penalty: 5
Via penalty: 10
